# Loading and cleaning the data

In [2]:
!pip install pandas
!pip install pyarrow
!pip install pandas_profiling

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [4]:
import numpy as np
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
from pandas_profiling import ProfileReport

In [5]:
df = pd.read_parquet('data/safety-SanFrancisco.parquet')

In [6]:
df.shape

(3958889, 12)

### Check the 5 first lines

In [7]:
df.head()

requestId dataType dataSubtype             dateTime  \
0          1   Safety    911_Fire  2018-10-02 11:54:01   
1          2   Safety     311_All  2018-07-08 15:00:27   
2          3   Safety     311_All  2016-06-28 13:12:28   
3          7   Safety     311_All  2017-03-03 09:34:49   
4         10   Safety    911_Fire  2016-10-25 18:33:20   

                         category  \
0    Potentially Life-Threatening   
1    Street and Sidewalk Cleaning   
2  General Request - COUNTY CLERK   
3          Temporary Sign Request   
4    Potentially Life-Threatening   

                                   subcategory  status  \
0                             Medical Incident    None   
1                                  Bulky Items  Closed   
2                            customer_callback  Closed   
3  Temporary Sign Request for Other_Event_Type  Closed   
4                             Medical Incident    None   

                                         address   latitude   longitude  \
0                             ELM ST/FRANKLIN ST  37.781286 -122.422187   
1   1536 SACRAMENTO ST, SAN FRANCISCO, CA, 94109  37.791887 -122.418188   
2         Not associated with a specific address   0.000000    0.000000   
3  2190 NORTH POINT ST, SAN FRANCISCO, CA, 94123  37.802853 -122.443245   
4                      200 Block of DALEWOOD WAY  37.737953 -122.456498   

           source extendedProperties  
0            None                     
1  Mobile/Open311                     
2           Phone                     
3           Phone                     
4            None

### Get a date randomly to review the types of events that occurred at this date

In [8]:
df.dtypes

requestId               int64
dataType               object
dataSubtype            object
dateTime               object
category               object
subcategory            object
status                 object
address                object
latitude              float64
longitude             float64
source                 object
extendedProperties     object
dtype: object

dateTime columns are an object, we need to convert it to filter by date

In [11]:
df['dateTime'] = pd.to_datetime(df['dateTime'])

In [13]:
# get on 2018/01/20 data
mask = df['dateTime'].dt.date == pd.to_datetime('2018-01-20')
df_s = df[mask]
df_s.head()

requestId dataType dataSubtype            dateTime  \
238         569   Safety    911_Fire 2018-01-20 18:15:55   
1047       2456   Safety    911_Fire 2018-01-20 21:49:34   
1508       3545   Safety    911_Fire 2018-01-20 08:29:22   
3252       7788   Safety     311_All 2018-01-20 11:23:21   
4538      10853   Safety    911_Fire 2018-01-20 22:47:09   

                          category       subcategory  status  \
238           Non Life-threatening  Medical Incident    None   
1047  Potentially Life-Threatening  Medical Incident    None   
1508  Potentially Life-Threatening  Medical Incident    None   
3252                   Sign Repair    Sign - Defaced  Closed   
4538  Potentially Life-Threatening  Medical Incident    None   

                                        address   latitude   longitude  \
238                         18TH ST/HARTFORD ST  37.760955 -122.433904   
1047                     200 Block of MARKET ST  37.792824 -122.397034   
1508                   2100 Block of THOMAS AVE  37.735738 -122.400562   
3252  392 GUERRERO ST, SAN FRANCISCO, CA, 94103  37.764869 -122.424401   
4538                      4000 Block of 18TH ST  37.760977 -122.434457   

              source extendedProperties  
238             None                     
1047            None                     
1508            None                     
3252  Mobile/Open311                     
4538            None

Check the volume of requests

In [14]:
df_s['dataSubtype'].value_counts()

311_All     1162
911_Fire     896
Name: dataSubtype, dtype: int64

In [15]:
df_s['category'].value_counts().head(10)

Street and Sidewalk Cleaning    420
Potentially Life-Threatening    414
Non Life-threatening            236
Graffiti                        233
Alarm                           210
Encampments                     128
Abandoned Vehicle                58
Fire                             36
MUNI Feedback                    36
Homeless Concerns                35
Name: category, dtype: int64

### Pick an address in the data and review its history of safety events

In [16]:
df_s['address'].value_counts().head(10)

200 Block of MARKET ST                    31
Not associated with a specific address    22
300 Block of EDDY ST                      20
800 Block of MARKET ST                    16
CASTRO ST/MARKET ST                       13
2900 Block of FOLSOM ST                   12
200 Block of EDDY ST                      12
1300 Block of CASTRO ST                   11
1100 Block of WEBSTER ST                  10
700 Block of FRANCISCO ST                 10
Name: address, dtype: int64

In [18]:
# Filtering
df[df['address']=='700 Block of FRANCISCO ST'].sort_values('dateTime').head()

requestId dataType dataSubtype            dateTime category  \
2439521    1119069   Safety    911_Fire 2016-03-01 15:03:13    Alarm   
1195321     501611   Safety    911_Fire 2016-03-01 15:03:13    Alarm   
1835982     854834   Safety    911_Fire 2016-03-01 15:03:13    Alarm   
1991697      31728   Safety    911_Fire 2016-03-01 15:03:13    Alarm   
2041366     152339   Safety    911_Fire 2016-03-01 15:03:13    Alarm   

            subcategory status                    address   latitude  \
2439521  Structure Fire   None  700 Block of FRANCISCO ST  37.804282   
1195321  Structure Fire   None  700 Block of FRANCISCO ST  37.804282   
1835982  Structure Fire   None  700 Block of FRANCISCO ST  37.804282   
1991697  Structure Fire   None  700 Block of FRANCISCO ST  37.804282   
2041366  Structure Fire   None  700 Block of FRANCISCO ST  37.804282   

          longitude source extendedProperties  
2439521 -122.417101   None                     
1195321 -122.417101   None                     
1835982 -122.417101   None                     
1991697 -122.417101   None                     
2041366 -122.417101   None

We need to drop duplicates except for the requestid column

## Identify potentiel data quality issues

In [19]:
# check the data type of each columns (now datetime type is correct)
df.dtypes

requestId                      int64
dataType                      object
dataSubtype                   object
dateTime              datetime64[ns]
category                      object
subcategory                   object
status                        object
address                       object
latitude                     float64
longitude                    float64
source                        object
extendedProperties            object
dtype: object

In [20]:
# display the number of unique values one colum at a time
for column in df.columns:
    print(column, df[column].nunique())

requestId 1185499
dataType 1
dataSubtype 2
dateTime 2613287
category 95
subcategory 639
status 2
address 226724
latitude 1206063
longitude 1159108
source 8
extendedProperties 1


In [24]:
# list requests to find missing columns (with percent)
(df.isna().sum(axis=0) / df.shape[0]).round(3) * 100

requestId              0.0
dataType               0.0
dataSubtype            0.0
dateTime               0.0
category               0.1
subcategory            0.0
status                35.5
address                0.0
latitude               0.0
longitude              0.0
source                35.5
extendedProperties     0.0
dtype: float64

In [25]:
# there's a strong balance of frequency
# 89% of cases happen for the first 15 categories out 
# of 90 === we won't able to model categories with little 
# volume acurrately
df['category'].value_counts()

Street and Sidewalk Cleaning               965968
Potentially Life-Threatening               676297
Non Life-threatening                       349634
Alarm                                      326893
Graffiti                                   296927
                                            ...  
General Request - STATUS OF WOMEN               6
General Request - LANGUAGE SERVICES             5
General Request - WAR MEMORIAL                  5
General Request - CONVENTION FACILITIES         1
General Request - GRANTS FOR THE ARTS           1
Name: category, Length: 95, dtype: int64

In [27]:
df['category'].value_counts().head(15).sum() / df.shape[0] * 100

89.04162253601957

## Generate a report

In [28]:
from pandas_profiling import ProfileReport

# take a random sample of 1000000 requests in the data to get faster
profile = ProfileReport(df.sample(100000, random_state=0))

In [29]:
profile

Summarize dataset:   0%|          | 0/26 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

In [31]:
# latitude and longitude columns have no missing values, 
# however they do have values set to 0, which here must have 
# the same meaning.

(df[['latitude', 'longitude']] == 0.0).sum(axis=0)

latitude     109599
longitude    109599
dtype: int64

## Store cleaned dataset to parquet

### Function to clean data

In [49]:
# remove columns that contain constant values (dataType and extendedProperties)
# convert dateTime to a datetime type
# convert columns that have less than a few 100 unique string as categorial data type to use less memory
# remove duplicated rows

def clean_raw_data(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    # convert dateTime to a datetime type
    df['dateTime'] = pd.to_datetime(df['dateTime'])
    
    # remove columns that contain constant values (dataType and extendedProperties)
    for col in ['dataType', 'extendedProperties']:
        del df[col]
    
    # convert columns that have less than a few 100 unique string as categorial data type to use less memory
    for col in ['dataSubtype', 'category', 'subcategory', 'source', 'status']:
        df[col] = pd.Categorical(df[col])
    
    # remove duplicated rows
    agg_columns = df.columns.tolist()
    agg_columns.remove('requestId')
    df = df.drop_duplicates(agg_columns)
    return df

### Call the function

In [50]:
df_p = clean_raw_data(df.copy())

In [51]:
# store the cleaned data
df_p.to_parquet('data/safety-SanFrancisco-1.parquet', compression='snappy')

## Examine NYC data

In [52]:
df_nyc = pd.read_parquet('data/safety-Nyc.parquet')

In [53]:
# same columns
print('SF', df.columns.tolist())
print('NYC', df_nyc.columns.tolist())

SF ['requestId', 'dataType', 'dataSubtype', 'dateTime', 'category', 'subcategory', 'status', 'address', 'latitude', 'longitude', 'source', 'extendedProperties']
NYC ['requestId', 'dataType', 'dataSubtype', 'dateTime', 'category', 'subcategory', 'status', 'address', 'latitude', 'longitude', 'source', 'extendedProperties']


In [54]:
# compare categories
categories = {
    'SF': df['category'].str.lower().unique(),
    'NYC': df_nyc['category'].str.lower().unique(),
}
                                   
for key, val in categories.items():
    print(f'{key}: number of unique categories {len(val)}')
                                   

SF: number of unique categories 96
NYC: number of unique categories 398


In [55]:
set(categories['SF']).intersection(categories['NYC'])

{'abandoned vehicle', 'graffiti'}

Different categories for cities, NYC has way more categories

In [57]:
from pathlib import Path

data_folder = Path('data')

for path_s in [
    'safety-Nyc.parquet',
    'safety-SanFrancisco.parquet'
]:
    path = data_folder / path_s
    print(f'Processing {path}...')
    df = pd.read_parquet(path)
    
    clean_raw_data(df).to_parquet(
        data_folder / path.name.replace('.parquet', '-1.parquet'),
        compression='snappy'
    )

Processing data/safety-Nyc.parquet...
Processing data/safety-SanFrancisco.parquet...
